# Setup

In [ ]:
# Import modules
import os,sys
import numpy as np
from datetime import datetime

# Add the directory containing 'cmct' to the Python path
# Navigate one level up to reach main CmCt dir
cmct_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
# Import utilities for this comparison
sys.path.insert(0,cmct_dir)

import cmct

import matplotlib as mpl
from matplotlib import pyplot as plt

In [ ]:
# Ice sheet
loc = 'GIS' # 'GIS' or 'AIS'
polar_stereographic = cmct.projection.set_projection(loc)

# GSFC mascons filename
gravimetry_gsfc_obs_filename = '/home/jovyan/shared-public/CmCt/datasets/gsfc.glb_.200204_202406_rl06v2.0_obp-ice6gd.h5'
shape_filename = '/home/jovyan/CmCt/data/ne_10m_coastline/ne_10m_coastline.shp'

# Set time range for comparison
start_date = '2003-01-15' #'2003-01-15'
end_date = '2024-12-31' #'2024-12-31' 

# IMBIE dataset filename
if loc == 'GIS': imbie_obs_filename = cmct_dir + '/data/IMBIE/imbie_greenland_2021_Gt.csv'
if loc == 'AIS': imbie_obs_filename = cmct_dir + '/data/IMBIE/imbie_antarctica_2021_Gt.csv'

# To save files, provide filenames. To turn saving off, set filenames to 'None'
plot_jpl_mascons_filename = 'mascons_jpl_' + loc + '.png'
plot_timeseries_filename = 'timeseries_gravimetry_and_imbie_obs_' + loc + '.png'


## Load JPL mascons

In [ ]:
!chmod 0400 ~/.netrc

edl = 'urs.earthdata.nasa.gov'
cmct.mascons.setup_earthdata_login_auth(edl)
print('Earthdata login credentials configured. Ready.')

In [ ]:
s3_bucket = 's3://podaac-ops-cumulus-protected/TELLUS_GRAC-GRFO_MASCON_CRI_GRID_RL06.3_V4'

# Load observation data(mascons)
ds_new, mascons_jpl = cmct.mascons.load_jpl_solution(s3_bucket)

# Compute the mascon means and calulate mass change of observation data
mass_change_obs_jpl, I_jpl_ = cmct.gravimetry.computeMasconMeans(mascons_jpl, start_date, end_date, loc)
time_timeseries_jpl, mass_delta_Gt_timeseries_jpl = cmct.gravimetry.calc_mass_delta_Gt_timeseries(mascons_jpl, start_date, end_date, I_jpl_)

# Convert times to years, months
years_jpl, months_jpl = cmct.time_utils.dt64_to_years_months(time_timeseries_jpl)


In [ ]:
# Check sum of mascon areas
# From Mike Croteau's Mascon Visualization Tool:
#  GIS area =  2189202.95
#  AIS area = 12425189.72

print('{: 10.2f}'.format(np.sum(mascons_jpl.areas[I_jpl_])))
if loc == 'GIS': print('{: 10.2f}'.format( 2189202.95))
if loc == 'AIS': print('{: 10.2f}'.format(12425189.72))

### Plot JPL mascons over the selected ice sheet

In [ ]:
import cartopy.crs as ccrs

i_0 = np.abs(ds_new['time_dt64'] - np.datetime64('2003-01-01T00:00:00.000000000')).argmin()
i_1 = np.abs(ds_new['time_dt64'] - np.datetime64('2024-12-31T00:00:00.000000000')).argmin()


plot_lons = np.arange(0.,360.25,.5)
plot_lats = np.arange(-90.,90.25,.5)

mascons_all = mascons_jpl.cmwe[:,i_1] - mascons_jpl.cmwe[:,i_0]
plot_grid = np.nan * np.ones(mascons_all.shape)
plot_grid[I_jpl_] = mascons_all[I_jpl_]
plot_grid = plot_grid.reshape(360,720)

plot_title = 'JPL Mascons (' + ds_new['time_dt64'][i_0].astype(str)[:10] + ' - ' + ds_new['time_dt64'][i_1].astype(str)[:10] + ')'
plot_units = 'cm w.e. / yr'
grid_min = -200
grid_max = +200

# Trend Plot
polar_stereographic = cmct.projection.set_projection(loc)

if loc == "GIS":
    extent = [-65, -20, 57, 84]
    resolution_value='10m'
elif loc == "AIS":
    extent = [-180, 180, -90, -65]
    resolution_value='110m'

fig = plt.figure(figsize=(8,8)) #, dpi=150)
ax = plt.axes(projection=polar_stereographic)
ax.set_extent(extent, crs=ccrs.PlateCarree())

ax.gridlines(zorder=5, linestyle=':', linewidth=0.5)
pc = ax.pcolormesh(plot_lons, plot_lats, plot_grid, cmap=plt.cm.Spectral, transform=ccrs.PlateCarree(), vmin=grid_min, vmax=grid_max)

c = plt.colorbar(pc, shrink=0.8)
c.ax.tick_params(labelsize=7)
c.set_label(plot_units, size=7)

ax.coastlines(resolution=resolution_value, zorder=10, linewidth=0.5)

plt.title(plot_title)

if plot_jpl_mascons_filename is not None: plt.savefig(plot_jpl_mascons_filename, bbox_inches='tight')


## Load GSFC mascons

In [ ]:
# Load observation data(mascons)
mascons_gsfc = cmct.gravimetry.loadGsfcMascons(gravimetry_gsfc_obs_filename)

# Compute the mascon means and calulate mass change of observation data
mass_change_obs_gsfc, I_gsfc_ = cmct.gravimetry.computeMasconMeans(mascons_gsfc, start_date, end_date, loc)
time_timeseries_gsfc, mass_delta_Gt_timeseries_gsfc = cmct.gravimetry.calc_mass_delta_Gt_timeseries(mascons_gsfc, start_date, end_date, I_gsfc_)

#cmct.gravimetry.plotFigure(mass_change_obs, mass_change_obs, mass_change_obs, mascons_gsfc, I_, start_date, end_date, polar_stereographic, loc, shape_filename, None)

# Convert times to years, months
years_gsfc, months_gsfc = cmct.time_utils.dt64_to_years_months(time_timeseries_gsfc)


## Load IMBIE

In [ ]:
start_date_fract = 1992.0
end_date_fract = 2024.0
mass_balance_column = 'Cumulative mass balance (Gt)'

# Read IMBIE mass change from start to end date
IMBIE_mass_change = cmct.imbie.process_imbie_data(imbie_obs_filename, start_date_fract, end_date_fract, mass_balance_column)

IMBIE_mass_change_annual = np.diff(IMBIE_mass_change['Cumulative mass balance (Gt)'][11::12].values)
IMBIE_mass_change_annual = np.insert(IMBIE_mass_change_annual, 0, IMBIE_mass_change['Cumulative mass balance (Gt)'][12])

IMBIE_mass_change_annual_cumulative = np.cumsum(IMBIE_mass_change_annual)


## Plot all time series

In [ ]:
colors = mpl.colormaps['tab10'].colors

# Get reference mass from start of JPL mascon timeseries
time_timeseries_jpl_ref = time_timeseries_jpl[0]
mass_delta_Gt_ref = mass_delta_Gt_timeseries_jpl[0]

# Get reference mass from start of JPL mascon timeseries
time_timeseries_jpl_ref = np.datetime64(time_timeseries_jpl[0])
mass_delta_Gt_ref = mass_delta_Gt_timeseries_jpl[0]

# Shift the IMBIE curve to match the reference mass
time_timeseries_IMBIE_dt = [np.datetime64(datetime(year, 1, 1)) for year in np.arange(1993,2021)]
shift = mass_delta_Gt_ref - np.interp(cmct.time_utils.dt64_to_fractional_year(time_timeseries_jpl_ref), \
          [cmct.time_utils.dt64_to_fractional_year(t) for t in time_timeseries_IMBIE_dt], \
          IMBIE_mass_change_annual_cumulative)

fig, ax = plt.subplots(2,1,figsize=(10,5))
ax[0].plot(time_timeseries_gsfc, mass_delta_Gt_timeseries_gsfc, '.-', c=colors[2], label='GSFC mascons')
ax[0].plot(time_timeseries_jpl,  mass_delta_Gt_timeseries_jpl,  '.-', c=colors[1], label='JPL mascons')
ax[0].plot(time_timeseries_IMBIE_dt, IMBIE_mass_change_annual_cumulative+shift, '.-', c=colors[0], label='IMBIE (annual)')

for year in range(2003,2024):
    idx = years_jpl == year
    ax[1].plot(months_jpl[idx], mass_delta_Gt_timeseries_jpl[idx]-mass_delta_Gt_timeseries_jpl[idx][2], '.-', c=[0.7, 0.7, 0.7], linewidth=0.3)
year = 2024
idx = years_jpl == year
ax[1].plot(months_jpl[idx], mass_delta_Gt_timeseries_jpl[idx]-mass_delta_Gt_timeseries_jpl[idx][1], '.-', c=colors[1], linewidth=2.0, markersize=10) #, label='JPL mascons')

ax[1].set_xticks(np.arange(1, 13))
ax[1].set_xticklabels(['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])

# Add axes for sea level equivalent
ax_sle = ax[0].secondary_yaxis("right")
ax_sle.set_yticks(np.array([-20, -15, -10, -5, 0, 5, 15]) * 361.8)
ax_sle.set_yticklabels(['{: 4.1f}'.format(-yt/361.8) for yt in ax_sle.get_yticks()])
ax_sle.set_ylabel('mm GMSL')
ax_sle = ax[1].secondary_yaxis("right")
ax_sle.set_yticks(np.array([-2.0, -1.5, -1.0, -0.5, 0.0, 0.5, 1.5]) * 361.8)
ax_sle.set_yticklabels(['{: 4.1f}'.format(-yt/361.8) for yt in ax_sle.get_yticks()])
ax_sle.set_ylabel('mm GMSL')

ax[0].legend()
#ax[1].legend()

ax[0].set_ylabel('Gt')
ax[1].set_ylabel('Gt')
ax[0].set_title(loc)

if plot_timeseries_filename is not None: plt.savefig(plot_timeseries_filename, bbox_inches='tight')


In [ ]:
# Debug checks:
# 1.) Check sum of mascon areas of Greenland and Antarctica in JPL mascons ... Greenland has extra mascons in Arctic Canada
# 2.) Do we need to remove mass change from peripheral GICs in JPL/GSFC mascon solutions?
# 3.) Plot full IMBIE timeseries or annual totals?

# Ideas:
# 1.) Rank 2024 in terms of annual change (loss or gain)
years = np.unique(years_jpl)
mass_annual_Gt_timeseries_jpl = np.nan * np.ones((len(years,)))
for i, year in enumerate(years):
    idx = years_jpl == year
    mass_annual_Gt_timeseries_jpl[i] = mass_delta_Gt_timeseries_jpl[idx][-1] - mass_delta_Gt_timeseries_jpl[idx][0]

year_biggest_loss = years[np.argmin(mass_annual_Gt_timeseries_jpl)]
print(year_biggest_loss, np.min(mass_annual_Gt_timeseries_jpl))

# 2.) Calculate sea-level contribution
# DONE!

# 3.) Maybe look at gain/loss over the year -> but would this make more sense if using balance years?
# 4.) Add error bars
# 5.) Do peripheral glaciers around GrIS need to be accounted for so that we don't double count?
# 6.) Separate into regional changes and attribute to atm/ocean patterns?
# 7.) ICESat-2 ???
# 8.) Can we make an interactive site for the web?

years = np.unique(years_jpl)
mass_annual_Gt_timeseries_jpl = np.nan * np.ones((len(years,)))
for i, year in enumerate(years):
    idx = years_jpl == year
    mass_annual_Gt_timeseries_jpl[i] = mass_delta_Gt_timeseries_jpl[idx][-1] - mass_delta_Gt_timeseries_jpl[idx][0]

year_biggest_loss = years[np.argmin(mass_annual_Gt_timeseries_jpl)]
print(year_biggest_loss, np.min(mass_annual_Gt_timeseries_jpl))
